In [1]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from utils import *
from Models import GRUModel
from Datasets import DatasetWithPlans
from CustomMSE import CustomMSE

# use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: " + str(device))

SUMO_HOME found
SUMO_HOME found
Using device: cuda


In [2]:
planned_path = get_planned_path()
path_values = list(planned_path.values())

 Retrying in 1 seconds
***Starting server on port 54031 ***
Loading net-file from './config/osm.net.xml.gz' ... done (114ms).
Loading done.
Simulation version 1.20.0 started with time: 0.00.
Simulation ended at time: 10777.00
Reason: TraCI requested termination.
Performance: 
 Duration: 8.64s
 TraCI-Duration: 5.03s
 Real time factor: 1247.92
 UPS: 135625.057897
Vehicles: 
 Inserted: 2199
 Running: 0
 Waiting: 0
Statistics (avg of 2199):
 RouteLength: 4984.74
 Speed: 9.48
 Duration: 532.63
 WaitingTime: 19.92
 TimeLoss: 76.98
 DepartDelay: 0.53



In [3]:
df = pd.read_csv('edinburgh_trajectories.csv')
num_cols = df.shape[1]
position_indices = [i for i in range(num_cols) if i % 4 == 1 or i % 4 == 2]
position_df = df.iloc[:, position_indices]
position_array = position_df.to_numpy()
sequence_length = len(position_indices) // 2
tensor_list = []

for row in position_array:
    reshaped_tensor = torch.tensor(row.reshape(sequence_length, 2))
    tensor_list.append(reshaped_tensor)

all_trajectories_tensor = torch.stack(tensor_list) / 10

# split the data into training and validation sets
# because the data is randomly generated, we don't need to shuffle it
train_ratio = 0.8
train_size = int(train_ratio * all_trajectories_tensor.shape[0])
train_trajectories_tensor = all_trajectories_tensor[:train_size]
val_trajectories_tensor = all_trajectories_tensor[train_size:]

train_path_values = torch.tensor(path_values[:train_size]) / 10
val_path_values = torch.tensor(path_values[train_size:]) / 10

train_mask = generate_masks(train_trajectories_tensor)

train_dataset = DatasetWithPlans(train_trajectories_tensor, train_mask, train_path_values)

In [6]:
def train_model(model, dataloader, epochs, optimizer, criterion):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, masks, planned_path in dataloader:
            optimizer.zero_grad()
            hidden = None
            loss = 0
            # Autoregressive prediction
            # Start with the first input and predict each subsequent step
            seq_len = inputs.size(1)
            current_input = inputs[:, 0, :].unsqueeze(1)
            for t in range(1, seq_len):
                prediction, hidden = model(current_input, hidden)

                previous_input = inputs[:, t-1, :]
                if epoch > 30:
                    if epoch > 40:
                        projected = project_to_nearest(prediction, planned_path)
                        if prediction.isnan().any():
                            break
                        current_input = (projected * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
                    else:
                        current_input = (prediction * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
                else:
                    current_input = inputs[:, t, :].unsqueeze(1)
                loss += criterion(prediction - previous_input, inputs[:, t, :]-previous_input) #(current_input-previous_input).squeeze(1))
                
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
 
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}')

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

model = GRUModel(input_size=2, hidden_size=128, num_layers=2, num_classes=2).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = CustomMSE()
train_model(model, train_dataloader, 50, optimizer, criterion)

Epoch 1, Loss: 75541922.28571428
Epoch 2, Loss: 69793078.85714285
Epoch 3, Loss: 59602178.71428572
Epoch 4, Loss: 46523253.85714286
Epoch 5, Loss: 32750182.5
Epoch 6, Loss: 21810644.285714287
Epoch 7, Loss: 15051108.42857143
Epoch 8, Loss: 12047828.857142856
Epoch 9, Loss: 10774259.82142857
Epoch 10, Loss: 7975708.767857143
Epoch 11, Loss: 6568854.285714285
Epoch 12, Loss: 5763774.919642857
Epoch 13, Loss: 5082229.366071428
Epoch 14, Loss: 4254422.258928572
Epoch 15, Loss: 3300602.339285714
Epoch 16, Loss: 2412273.96875
Epoch 17, Loss: 1641409.15625
Epoch 18, Loss: 1110670.2254464286
Epoch 19, Loss: 778264.8504464285
Epoch 20, Loss: 567689.0457589285
Epoch 21, Loss: 446290.0479910714
Epoch 22, Loss: 365886.41015625
Epoch 23, Loss: 310524.96875
Epoch 24, Loss: 274993.82198660716
Epoch 25, Loss: 247191.19419642858
Epoch 26, Loss: 227613.32700892858
Epoch 27, Loss: 217819.09095982142
Epoch 28, Loss: 210468.79185267858
Epoch 29, Loss: 199691.00111607142
Epoch 30, Loss: 196827.95982142858
E

In [7]:
def checkModel(model, inputs, masks, paths):
    model.eval()
    with torch.no_grad():
        hidden = None
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            prediction, hidden = model(current_input, hidden)
            projected = project_to_nearest(prediction, paths)
            current_input = (projected * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            print(projected[2], inputs[:, t, :][2])

# get the first batch in dataloader
val_mask = generate_masks(val_trajectories_tensor, missing_ratio=0.9, complete_traj_ratio=0)
val_dataset = DatasetWithPlans(val_trajectories_tensor, val_mask, val_path_values)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
inputs, masks, paths = next(iter(val_dataloader))

checkModel(model, inputs, masks, paths)

tensor([176.2100, 356.4040], device='cuda:0') tensor([176.2920, 355.7153], device='cuda:0')
tensor([176.2100, 356.4040], device='cuda:0') tensor([176.3395, 355.3162], device='cuda:0')
tensor([176.6323, 352.8561], device='cuda:0') tensor([176.4043, 354.7715], device='cuda:0')
tensor([176.5952, 353.1674], device='cuda:0') tensor([176.4971, 353.9917], device='cuda:0')
tensor([176.7490, 351.8757], device='cuda:0') tensor([176.6195, 352.9636], device='cuda:0')
tensor([176.9498, 350.1883], device='cuda:0') tensor([176.7493, 351.8733], device='cuda:0')
tensor([177.1575, 348.4433], device='cuda:0') tensor([176.8926, 350.6688], device='cuda:0')
tensor([177.3528, 346.8024], device='cuda:0') tensor([177.0351, 349.4719], device='cuda:0')
tensor([177.5333, 345.2859], device='cuda:0') tensor([177.1749, 348.2968], device='cuda:0')
tensor([177.6989, 343.8948], device='cuda:0') tensor([177.3130, 347.1373], device='cuda:0')
tensor([177.5748, 344.9377], device='cuda:0') tensor([177.4436, 346.0396], devic

In [8]:
def evaluate_model(model, dataloader):
    model.eval()
    total_loss = 0
    steps = 0
    for inputs, masks, paths in dataloader:
        hidden = None
        loss = 0
        seq_len = inputs.size(1)
        current_input = inputs[:, 0, :].unsqueeze(1)
        for t in range(1, seq_len):
            new_steps = int(sum((inputs[:, t, :].reshape(-1) != 0).float().to(device))) / 2
            if new_steps == 0:
                break
            steps += new_steps
            prediction, hidden = model(current_input, hidden)
            projected = project_to_nearest(prediction, paths)
            current_input = (projected * masks[:, t, :] + inputs[:, t, :] * (1-masks[:, t, :])).unsqueeze(1)
            loss += torch.norm((projected - inputs[:, t, :]) * 10, dim=1) * (inputs[:, t, :] != 0)[:, 0]
        total_loss += loss.sum().item()
    print(f'Validation Loss: {total_loss / steps}')

for i in [0.9, 0.8, 0.7, 0.6, 0.5]:
    val_mask = generate_masks(val_trajectories_tensor, missing_ratio=i, complete_traj_ratio=0)
    val_dataset = DatasetWithPlans(val_trajectories_tensor, val_mask, val_path_values)
    val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
    evaluate_model(model, val_dataloader)

Validation Loss: 120.73312230395105
Validation Loss: 69.53732899960472
Validation Loss: 50.198508150445974
Validation Loss: 39.43855908536271
Validation Loss: 32.70856194253012
